<a href="https://colab.research.google.com/github/AdaUchendu/TuringBench/blob/master/BertAA_for_TuringBench.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BertAA: BERT fine-tuning for Authorship Attribution**



---

**Paper**: https://www.overleaf.com/6941722415tfbcbyzmxvsh

---

Authors: Maël Fabien, Esaú Villatoro-Tello, Petr Motlicek, Shantipriya Parida

*Idiap Research Institute, Martigny, Switzerland*

*Ecole Polytechnique Fédérale de Lausanne (EPFL), Switzerland*

# I. Task description

## Main idea:
- Train an authorship attribution algorithm on Enron E-mail corpus, IMDB movie reviews and Blog Authroship corpus
- Compare several approaches:
  - TF-IDF + Classifier
  - Manual feature extraction + Classifier
  - Pre-trained Bert + Classifier
  - Additional layer on pre-trained Bert
- Compare the performance on 5, 10, 25, 50, 75 and 100 characters

Datasets can be downloaded here:
- IMDb and IMDb 62: https://umlt.infotech.monash.edu/?page_id=266
- Blog: https://www.kaggle.com/rtatman/blog-authorship-corpus/data#
- Enron: Preparation done by LUH

## Papers on Enron - Summary:

| Paper | Approach | Result | Year|
| --- | --- | --- |--- |
|[An Exploratory Study on Authorship Verification Models for Forensic Purpose](https://repository.tudelft.nl/islandora/object/uuid:f43d489f-c53c-4392-824b-e1a8eaebb827/datastream/OBJ/download) | Master thesis - N-Gram (word & char) - 38 authors| Best Accuracy: 0.73 | 2013 |
| [Authorship Attribution on the Enron Email Corpus](https://dsc.duq.edu/cgi/viewcontent.cgi?article=1839&context=etd) | Master Thesis - N-Gram (word & char) - Several author sizes | Best Accuracy: 0.5353| 2013 |
| [Authorship Attribution of E-Mail: Comparing Classifiers Over a New Corpus for Evaluation](http://www.lrec-conf.org/proceedings/lrec2008/pdf/552_paper.pdf) | N-gram (Word), stemming - Hierarchical Prob. Clf - 9 authors| Best Accuracy: 0.8705| 2008 |
| [Authorship Verification for Short Messages using Stylometry](https://www.uvic.ca/engineering/ece/isot/assets/docs/Authorship_Verification_for_Short_Messages_using_Stylometry.pdf) | N-gram (absent, present, not freq) - 87 authors| EER 14.35% |2013 |
| [E-Mail Authorship Attribution applied to the Extended Enron Authorship Corpus (XEAC)](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.466.4644&rep=rep1&type=pdf) | Custom style-feature extraction| Best accuracy: 0.8922 | 2009|
| [Identifying idiolect in forensic authorship attribution: an n-gram textbite approach](http://irep.ntu.ac.uk/id/eprint/15374/1/220199_PubSub2404_Wright.pdf) | 1 author vs. rest - N-Grams| Best accuracy: 100% (Not same task) | |
| [CEAI: CCM-based email authorship identification model](https://www.sciencedirect.com/science/article/pii/S111086651300039X) | Stylometric + Content features | 94% - 10 aut., 89% - 25 aut., 81% - 50 aut.| 2013 |

## Papers on techniques (Not Enron) - Summary:

| Paper | Approach | Result | Year|
| --- | --- | --- |--- |
| [Siamese Networks for Large-Scale Author Identification](https://www.groundai.com/project/siamese-networks-for-large-scale-author-identification/1) | Siamese networks based on Bert | 94% - 2 authors, 40% - 50 authors | 2019 |
| [Topic or Style? Exploring the Most Useful Features for Authorship Attribution](https://www.aclweb.org/anthology/C18-1029.pdf) | Feature extraction process | |2018 |
| [Learning Invariant Representations of Social Media Users](https://www.aclweb.org/anthology/D19-1178.pdf) | Learn Embeddings with an Angular Margin | Works for unseen users (similarity between user styles) | 2019 |
| [Authorship Attribution for Forensic Investigation with Thousands of Authors](https://link.springer.com/content/pdf/10.1007%2F978-3-642-55415-5_28.pdf) | Manual features + gender and age classifier | Reaching 39% on thousands of authors | 2014 |
| [Convolutional Neural Networks for Authorship Attribution of Short Texts](https://www.aclweb.org/anthology/E17-2106.pdf) | Character N-gram + CNN + Softmax | 50% on 100 tweet authors | 2017 |





## Other information

- List of potential conferences to publish to: http://www.wikicfp.com/cfp/call?conference=NLP

- **Dataset**: the dataset has been already saved on Google Drive, and is available publically using the lines of code provided below.


**NB**: Imports due to some installs take on average 10 minutes to run.

In [ ]:
# General
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import itertools
from pandas import DataFrame

# Visualization
import matplotlib.font_manager as fm
from matplotlib.collections import QuadMesh
import seaborn as sn
import plotly.express as px

# Feature extraction approach
from sklearn.feature_extraction.text import TfidfVectorizer
from string import punctuation
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

# Classification
import xgboost as xgb
import lightgbm as lgbm
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# BERT classifier
# Installing a custom version of Simple Transformers
!git clone https://github.com/NVIDIA/apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex
#!git init
!pip install --upgrade tqdm
#!git remote add origin https://github.com/ThilinaRajapakse/simpletransformers.git
#!git pull origin master
##!pip install -r requirements-dev.txt
!pip install transformers
!pip install tensorboardX

!pip install simpletransformers
from simpletransformers.classification import ClassificationModel

import torch

# Access to the file
!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Parallelize apply on Pandas
!pip install pandarallel
from pandarallel import pandarallel
pandarallel.initialize()

# Evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
fatal: destination path 'apex' already exists and is not an empty directory.
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-l6_aq1eu
Created temporary directory: /tmp/pip-req-tracker-pk2p8oz3
Created requirements tracker '/tmp/pip-req-tracker-pk2p8oz3'
Created temporary directory: /tmp/pip-install-a9y6adtd
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-9hsqwett
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-pk2p8oz3'
    Running setup.py (path:/tmp/pip-req-build-9hsqwett/s

Load Data

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

In [ ]:
train = load_dataset('turingbench/TuringBench',name='AA', split='train')
train = pd.DataFrame.from_dict(train)

test = load_dataset('turingbench/TuringBench',name='AA', split='test')
test = pd.DataFrame.from_dict(test)

valid = load_dataset('turingbench/TuringBench',name='AA', split='valid')
valid = pd.DataFrame.from_dict(valid)

The code below is used to plot the confusion matrices during training. It is adapted from [here](https://github.com/wcipriano/pretty-print-confusion-matrix).

In [ ]:
def get_new_fig(fn, figsize=[9,9]):
    """ Init graphics """
    fig1 = plt.figure(fn, figsize)
    ax1 = fig1.gca()   #Get Current Axis
    ax1.cla() # clear existing plot
    return fig1, ax1

def configcell_text_and_colors(array_df, lin, col, oText, facecolors, posi, fz, fmt, show_null_values=0):
    """
      config cell text and colors
      and return text elements to add and to dell
      @TODO: use fmt
    """
    text_add = []; text_del = [];
    cell_val = array_df[lin][col]
    tot_all = array_df[-1][-1]
    per = (float(cell_val) / tot_all) * 100
    curr_column = array_df[:,col]
    ccl = len(curr_column)

    #last line  and/or last column
    if(col == (ccl - 1)) or (lin == (ccl - 1)):
        #tots and percents
        if(cell_val != 0):
            if(col == ccl - 1) and (lin == ccl - 1):
                tot_rig = 0
                for i in range(array_df.shape[0] - 1):
                    tot_rig += array_df[i][i]
                per_ok = (float(tot_rig) / cell_val) * 100
            elif(col == ccl - 1):
                tot_rig = array_df[lin][lin]
                per_ok = (float(tot_rig) / cell_val) * 100
            elif(lin == ccl - 1):
                tot_rig = array_df[col][col]
                per_ok = (float(tot_rig) / cell_val) * 100
            per_err = 100 - per_ok
        else:
            per_ok = per_err = 0

        per_ok_s = ['%.2f%%'%(per_ok), '100%'] [per_ok == 100]

        #text to DEL
        text_del.append(oText)

        #text to ADD
        font_prop = fm.FontProperties(weight='bold', size=fz)
        text_kwargs = dict(color='w', ha="center", va="center", gid='sum', fontproperties=font_prop)
        lis_txt = ['%d'%(cell_val), per_ok_s, '%.2f%%'%(per_err)]
        lis_kwa = [text_kwargs]
        dic = text_kwargs.copy(); dic['color'] = 'g'; lis_kwa.append(dic);
        dic = text_kwargs.copy(); dic['color'] = 'r'; lis_kwa.append(dic);
        lis_pos = [(oText._x, oText._y-0.3), (oText._x, oText._y), (oText._x, oText._y+0.3)]
        for i in range(len(lis_txt)):
            newText = dict(x=lis_pos[i][0], y=lis_pos[i][1], text=lis_txt[i], kw=lis_kwa[i])
            #print 'lin: %s, col: %s, newText: %s' %(lin, col, newText)
            text_add.append(newText)
        #print '\n'

        #set background color for sum cells (last line and last column)
        carr = [0.27, 0.30, 0.27, 1.0]
        if(col == ccl - 1) and (lin == ccl - 1):
            carr = [0.17, 0.20, 0.17, 1.0]
        facecolors[posi] = carr

    else:
        if(per > 0):
            txt = '%s\n%.2f%%' %(cell_val, per)
        else:
            if(show_null_values == 0):
                txt = ''
            elif(show_null_values == 1):
                txt = '0'
            else:
                txt = '0\n0.0%'
        oText.set_text(txt)

        #main diagonal
        if(col == lin):
            #set color of the textin the diagonal to white
            oText.set_color('w')
            # set background color in the diagonal to blue
            facecolors[posi] = [0.35, 0.8, 0.55, 1.0]
        else:
            oText.set_color('r')

    return text_add, text_del

def insert_totals(df_cm):
    """ insert total column and line (the last ones) """
    sum_col = []
    for c in df_cm.columns:
        sum_col.append( df_cm[c].sum() )
    sum_lin = []
    for item_line in df_cm.iterrows():
        sum_lin.append( item_line[1].sum() )
    df_cm['sum_lin'] = sum_lin
    sum_col.append(np.sum(sum_lin))
    df_cm.loc['sum_col'] = sum_col

def pretty_plot_confusion_matrix(df_cm, annot=True, cmap="Oranges", fmt='.2f', fz=11,
      lw=0.5, cbar=False, figsize=[8,8], show_null_values=0, pred_val_axis='y'):
    """
      print conf matrix with default layout (like matlab)
      params:
        df_cm          dataframe (pandas) without totals
        annot          print text in each cell
        cmap           Oranges,Oranges_r,YlGnBu,Blues,RdBu, ... see:
        fz             fontsize
        lw             linewidth
        pred_val_axis  where to show the prediction values (x or y axis)
                        'col' or 'x': show predicted values in columns (x axis) instead lines
                        'lin' or 'y': show predicted values in lines   (y axis)
    """
    if(pred_val_axis in ('col', 'x')):
        xlbl = 'Predicted'
        ylbl = 'Actual'
    else:
        xlbl = 'Actual'
        ylbl = 'Predicted'
        df_cm = df_cm.T

    # create "Total" column
    insert_totals(df_cm)

    #this is for print allways in the same window
    fig, ax1 = get_new_fig('Conf matrix default', figsize)

    #thanks for seaborn
    ax = sn.heatmap(df_cm, annot=annot, annot_kws={"size": fz}, linewidths=lw, ax=ax1,
                    cbar=cbar, cmap=cmap, linecolor='w', fmt=fmt)

    #set ticklabels rotation
    ax.set_xticklabels(ax.get_xticklabels(), rotation = 45, fontsize = 10)
    ax.set_yticklabels(ax.get_yticklabels(), rotation = 25, fontsize = 10)

    # Turn off all the ticks
    for t in ax.xaxis.get_major_ticks():
        t.tick1On = False
        t.tick2On = False
    for t in ax.yaxis.get_major_ticks():
        t.tick1On = False
        t.tick2On = False

    #face colors list
    quadmesh = ax.findobj(QuadMesh)[0]
    facecolors = quadmesh.get_facecolors()

    #iter in text elements
    array_df = np.array( df_cm.to_records(index=False).tolist() )
    text_add = []; text_del = [];
    posi = -1 #from left to right, bottom to top.
    for t in ax.collections[0].axes.texts: #ax.texts:
        pos = np.array( t.get_position()) - [0.5,0.5]
        lin = int(pos[1]); col = int(pos[0]);
        posi += 1
        #print ('>>> pos: %s, posi: %s, val: %s, txt: %s' %(pos, posi, array_df[lin][col], t.get_text()))

        #set text
        txt_res = configcell_text_and_colors(array_df, lin, col, t, facecolors, posi, fz, fmt, show_null_values)

        text_add.extend(txt_res[0])
        text_del.extend(txt_res[1])

    #remove the old ones
    for item in text_del:
        item.remove()
    #append the new ones
    for item in text_add:
        ax.text(item['x'], item['y'], item['text'], **item['kw'])

    #titles and legends
    ax.set_title('Confusion matrix')
    ax.set_xlabel(xlbl)
    ax.set_ylabel(ylbl)
    plt.tight_layout()  #set layout slim
    plt.show()

def plot_confusion_matrix_from_data(y_test, predictions, columns=None, annot=True, cmap="Oranges",
      fmt='.2f', fz=11, lw=0.5, cbar=False, figsize=[8,8], show_null_values=0, pred_val_axis='lin'):
    """
        plot confusion matrix function with y_test (actual values) and predictions (predic),
        whitout a confusion matrix yet
    """
    from sklearn.metrics import confusion_matrix
    from pandas import DataFrame

    #data
    if(not columns):
        #labels axis integer:
        ##columns = range(1, len(np.unique(y_test))+1)
        #labels axis string:
        from string import ascii_uppercase
        columns = ['class %s' %(i) for i in list(ascii_uppercase)[0:len(np.unique(y_test))]]

    confm = confusion_matrix(y_test, predictions)
    cmap = 'Oranges';
    fz = 11;
    figsize=[9,9];
    show_null_values = 2
    df_cm = DataFrame(confm, index=columns, columns=columns)
    pretty_plot_confusion_matrix(df_cm, fz=fz, cmap=cmap, figsize=figsize, show_null_values=show_null_values, pred_val_axis=pred_val_axis)


# III. Compute corpus hardness

Let us now characterize the hardness of the corpora. We implemented the Relative Hardness score, as defined here: https://dl.acm.org/doi/10.5555/1776334.1776358

In [ ]:
# Text pre-processing as required to compare the content of different authors
def fil_sent(sent):
    """
    Filter stopwords
    """
    filtered_sentence = ' '.join([w for w in sent.split() if not w in stop_words])
    filtered_sentence = ''.join([w for w in filtered_sentence if w not in list(punctuation)])
    filtered_sentence = filtered_sentence.strip()
    filtered_sentence = filtered_sentence.split()
    return filtered_sentence

def process(sent):
    """
    Apply stemming
    """
    sent = str(sent)
    return fil_sent(' '.join([ps.stem(str(x).lower()) for x in word_tokenize(sent)]))

stop_words = set(stopwords.words('english'))
ps = PorterStemmer() 

In [ ]:
# Similarity between two authors' content
def jaccard_similarity(list1, list2):

    list1 = list1[:512]
    list2 = list2[:512]
    
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection

    if union > 0:
      return float(intersection) / union
    else:
      return 0

In [ ]:
# Combines all the text per author in a list of lists
def all_text_all_category(df, col_id, col_text):
  all_text = []

  for val in list(set(df[col_id].values)):
    sub_df = df[df[col_id] == val]
    all_text.append(list(itertools.chain(sub_df[col_text].apply(lambda x: process(x)).values))[0])

  return all_text

In [ ]:
# Computes the relative hardness
def rel_hardness(df, col_id, col_text):

  n_labels = len(list(set(df[col_text].values)))
  return 1/(n_labels * (n_labels - 1) * 0.5) * sum([jaccard_similarity(x[0], x[1]) for x in itertools.combinations(all_text_all_category(df, col_id, col_text), 2) if x[0] != x[1]])

# IV. Core function

The function below:
- processes the dataframe and extracts features needed by some methods
- runs for 5, 10, 25, 50, 75 and 100 authors:
  - a logistic regression classifier on top of TF-IDF + Preprocessing
  - WhoBert
  - Style-only classifier
  - WhoBert Style
- returns the output performance and a graph

In [ ]:
# This is needed in case you want to try the Longtransformer for longer texts (we did not obtain better results)
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification

# Long-sequences for Blog authorship attribution
tokenizer = AutoTokenizer.from_pretrained('allenai/longformer-base-4096')
config = AutoConfig.from_pretrained('allenai/longformer-base-4096',num_labels=5)
model_long = AutoModelForSequenceClassification.from_pretrained('allenai/longformer-base-4096',config=config)

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias'

On Enron, the following function takes ± 6 hours to run

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.metrics import classification_report as cr

In [ ]:
def run_iterations(source="enron", recompute = False):

  # Load data and remove emails containing the sender's name

  print("Loading and processing dataframe")

  ## Pre-processing functions
  def is_name_in_email(name, email):
    """
    Removing emails from Enron where name is in email
    """

    if str(name).lower() in str(email).lower():
      return 1
    else:
      return 0

  def fil_sent(sent):
      """
      Filter stopwords
      """

      filtered_sentence = ' '.join([w for w in sent.split() if not w in stop_words])
      return filtered_sentence

  def process(sent):
      """
      Apply stemming
      """

      sent = str(sent)
      return fil_sent(' '.join([ps.stem(str(x).lower()) for x in word_tokenize(sent)]))

  stop_words = set(stopwords.words('english'))
  ps = PorterStemmer() 

  def extract_style(text):
    """
    Extracting stylometric features of a text
    """

    text = str(text)
    len_text = len(text)
    len_words = len(text.split())
    avg_len = np.mean([len(t) for t in text.split()])
    num_short_w = len([t for t in text.split() if len(t) < 3])
    per_digit = sum(t.isdigit() for t in text)/len(text)
    per_cap = sum(1 for t in text if t.isupper())/len(text)
    f_a = sum(1 for t in text if t.lower() == "a")/len(text)
    f_b = sum(1 for t in text if t.lower() == "b")/len(text)
    f_c = sum(1 for t in text if t.lower() == "c")/len(text)
    f_d = sum(1 for t in text if t.lower() == "d")/len(text)
    f_e = sum(1 for t in text if t.lower() == "e")/len(text)
    f_f = sum(1 for t in text if t.lower() == "f")/len(text)
    f_g = sum(1 for t in text if t.lower() == "g")/len(text)
    f_h = sum(1 for t in text if t.lower() == "h")/len(text)
    f_i = sum(1 for t in text if t.lower() == "i")/len(text)
    f_j = sum(1 for t in text if t.lower() == "j")/len(text)
    f_k = sum(1 for t in text if t.lower() == "k")/len(text)
    f_l = sum(1 for t in text if t.lower() == "l")/len(text)
    f_m = sum(1 for t in text if t.lower() == "m")/len(text)
    f_n = sum(1 for t in text if t.lower() == "n")/len(text)
    f_o = sum(1 for t in text if t.lower() == "o")/len(text)
    f_p = sum(1 for t in text if t.lower() == "p")/len(text)
    f_q = sum(1 for t in text if t.lower() == "q")/len(text)
    f_r = sum(1 for t in text if t.lower() == "r")/len(text)
    f_s = sum(1 for t in text if t.lower() == "s")/len(text)
    f_t = sum(1 for t in text if t.lower() == "t")/len(text)
    f_u = sum(1 for t in text if t.lower() == "u")/len(text)
    f_v = sum(1 for t in text if t.lower() == "v")/len(text)
    f_w = sum(1 for t in text if t.lower() == "w")/len(text)
    f_x = sum(1 for t in text if t.lower() == "x")/len(text)
    f_y = sum(1 for t in text if t.lower() == "y")/len(text)
    f_z = sum(1 for t in text if t.lower() == "z")/len(text)
    f_1 = sum(1 for t in text if t.lower() == "1")/len(text)
    f_2 = sum(1 for t in text if t.lower() == "2")/len(text)
    f_3 = sum(1 for t in text if t.lower() == "3")/len(text)
    f_4 = sum(1 for t in text if t.lower() == "4")/len(text)
    f_5 = sum(1 for t in text if t.lower() == "5")/len(text)
    f_6 = sum(1 for t in text if t.lower() == "6")/len(text)
    f_7 = sum(1 for t in text if t.lower() == "7")/len(text)
    f_8 = sum(1 for t in text if t.lower() == "8")/len(text)
    f_9 = sum(1 for t in text if t.lower() == "9")/len(text)
    f_0 = sum(1 for t in text if t.lower() == "0")/len(text)
    f_e_0 = sum(1 for t in text if t.lower() == "!")/len(text)
    f_e_1 = sum(1 for t in text if t.lower() == "-")/len(text)
    f_e_2 = sum(1 for t in text if t.lower() == ":")/len(text)
    f_e_3 = sum(1 for t in text if t.lower() == "?")/len(text)
    f_e_4 = sum(1 for t in text if t.lower() == ".")/len(text)
    f_e_5 = sum(1 for t in text if t.lower() == ",")/len(text)
    f_e_6 = sum(1 for t in text if t.lower() == ";")/len(text)
    f_e_7 = sum(1 for t in text if t.lower() == "'")/len(text)
    f_e_8 = sum(1 for t in text if t.lower() == "/")/len(text)
    f_e_9 = sum(1 for t in text if t.lower() == "(")/len(text)
    f_e_10 = sum(1 for t in text if t.lower() == ")")/len(text)
    f_e_11 = sum(1 for t in text if t.lower() == "&")/len(text)
    richness = len(list(set(text.split())))/len(text.split())

    return pd.Series([avg_len, len_text, len_words, num_short_w, per_digit, per_cap, f_a, f_b, f_c, f_d, f_e, f_f, f_g, f_h, f_i, f_j, f_k, f_l, f_m, f_n, f_o, f_p, f_q, f_r, f_s, f_t, f_u, f_v, f_w, f_x, f_y, f_z, f_0, f_1, f_2, f_3, f_4, f_5, f_6, f_7, f_8, f_9, f_e_0, f_e_1, f_e_2, f_e_3, f_e_4, f_e_5, f_e_6, f_e_7, f_e_8, f_e_9, f_e_10, f_e_11, richness])

  list_senders = [20, 5] #[5, 10, 25, 50, 75, 100]

  if source == "enron":

    if recompute:
      df = pd.read_csv('enron.csv')
      df['name'] = df['From'].apply(lambda x: x.split("'")[1].split(".")[0])
      df['name_in_mail'] = df.apply(lambda x: is_name_in_email(x['name'], x['content']), axis=1)
      df = df[df['name_in_mail'] == 0]
      df = df[df['content'].apply(lambda x: '-----' not in str(x))]
      df = df[df['content'].apply(lambda x: "@" not in str(x))]
      df = df[df['content'].apply(lambda x: "From: " not in str(x))]
      df = df[df['content'].apply(lambda x: len(str(x).split()) > 10)]
      df['content_tfidf'] = df['content'].parallel_apply(lambda x: process(x))
      df[["avg_len", "len_text", "len_words", "num_short_w", "per_digit", "per_cap", "f_a", "f_b", "f_c", "f_d", "f_e", "f_f", "f_g", "f_h", "f_i", "f_j", "f_k", "f_l", "f_m", "f_n", "f_o", "f_p", "f_q", "f_r", "f_s", "f_t", "f_u", "f_v", "f_w", "f_x", "f_y", "f_z", "f_0", "f_1", "f_2", "f_3", "f_4", "f_5", "f_6", "f_7", "f_8", "f_9",  "f_e_0", "f_e_1", "f_e_2", "f_e_3", "f_e_4", "f_e_5", "f_e_6", "f_e_7", "f_e_8", "f_e_9", "f_e_10", "f_e_11", "richness"]] = df['content'].parallel_apply(lambda x: extract_style(x))
      df.to_csv("full_enron.csv")
    else:
      df = pd.read_csv('full_enron.csv')
      df['name'] = df['From'].apply(lambda x: x.split("'")[1].split(".")[0])
      df['name_in_mail'] = df.apply(lambda x: is_name_in_email(x['name'], x['content']), axis=1)
      df = df[df['name_in_mail'] == 0]
      df = df[df['content'].apply(lambda x: '-----' not in str(x))]
      df = df[df['content'].apply(lambda x: "@" not in str(x))]
      df = df[df['content'].apply(lambda x: "From: " not in str(x))]
      df = df[df['content'].apply(lambda x: len(str(x).split()) > 10)]
      df.to_csv("full_enron2.csv")
      print(df.shape)

  elif source == "imdb":

    if recompute:
      df = pd.read_csv('full_imdb.csv', index_col = 0)
      df['content_tfidf'] = df['content'].parallel_apply(lambda x: process(x))
      df[["avg_len", "len_text", "len_words", "num_short_w", "per_digit", "per_cap", "f_a", "f_b", "f_c", "f_d", "f_e", "f_f", "f_g", "f_h", "f_i", "f_j", "f_k", "f_l", "f_m", "f_n", "f_o", "f_p", "f_q", "f_r", "f_s", "f_t", "f_u", "f_v", "f_w", "f_x", "f_y", "f_z", "f_0", "f_1", "f_2", "f_3", "f_4", "f_5", "f_6", "f_7", "f_8", "f_9",  "f_e_0", "f_e_1", "f_e_2", "f_e_3", "f_e_4", "f_e_5", "f_e_6", "f_e_7", "f_e_8", "f_e_9", "f_e_10", "f_e_11", "richness"]] = df['content'].parallel_apply(lambda x: extract_style(x))
      df.to_csv("full_imdb_feat.csv")
    else:
      df = pd.read_csv('full_imdb_feat.csv', index_col = 0)

  elif source == "imdb62":
    
    list_senders = [62]

    if recompute:

      df = pd.read_csv("imdb62.txt", sep="\t", header = None)
      df.columns = ["reviewid", "From", "Itemid", "grade", "title", "content"]
      df['content_tfidf'] = df['content'].parallel_apply(lambda x: process(x))
      df[["avg_len", "len_text", "len_words", "num_short_w", "per_digit", "per_cap", "f_a", "f_b", "f_c", "f_d", "f_e", "f_f", "f_g", "f_h", "f_i", "f_j", "f_k", "f_l", "f_m", "f_n", "f_o", "f_p", "f_q", "f_r", "f_s", "f_t", "f_u", "f_v", "f_w", "f_x", "f_y", "f_z", "f_0", "f_1", "f_2", "f_3", "f_4", "f_5", "f_6", "f_7", "f_8", "f_9",  "f_e_0", "f_e_1", "f_e_2", "f_e_3", "f_e_4", "f_e_5", "f_e_6", "f_e_7", "f_e_8", "f_e_9", "f_e_10", "f_e_11", "richness"]] = df['content'].parallel_apply(lambda x: extract_style(x))
      df.to_csv("full_imdb62.csv")

    else:
      
      df = pd.read_csv('full_imdb62.csv', index_col = 0)

  elif source == "blog":

    if recompute:
      df = pd.read_csv('/content/drive/MyDrive/Data/blogtext.csv')
      df.columns = ["From", "Gender", "Age", "Topic", "Sign", "Date", "content"]
      df = df[df['content'].apply(lambda x: len(x.split())) > 0]
      df['content_tfidf'] = df['content'].parallel_apply(lambda x: process(x))
      df[["avg_len", "len_text", "len_words", "num_short_w", "per_digit", "per_cap", "f_a", "f_b", "f_c", 
          "f_d", "f_e", "f_f", "f_g", "f_h", "f_i", "f_j", "f_k", "f_l", "f_m", "f_n", "f_o", "f_p", "f_q", 
          "f_r", "f_s", "f_t", "f_u", "f_v", "f_w", "f_x", "f_y", "f_z", "f_0", "f_1", "f_2", "f_3", "f_4", 
          "f_5", "f_6", "f_7", "f_8", "f_9",  "f_e_0", "f_e_1", "f_e_2", "f_e_3", "f_e_4", "f_e_5", "f_e_6", 
          "f_e_7", "f_e_8", "f_e_9", "f_e_10", "f_e_11", "richness"]] = df['content'].parallel_apply(lambda x: extract_style(x))
      df.to_csv("full_blog.csv")
    else:
      df = pd.read_csv('full_blog.csv')
      # Keep longer texts
      #df = df[df['content'].apply(lambda x: len(str(x).split()) > 30)]

  elif source == "AA":

    if recompute:
      df = pd.read_csv('/content/drive/MyDrive/TuringBench_Data/newdata.csv')
      df = df.rename(columns={'Generation': 'content', 'label': 'From'})
      df = df[['content', 'From']]
      df.columns = ["content", "From"]
      # df = df.sample(10)
      df = df[df['content'].apply(lambda x: len(str(x).split())) > 0]
      df['content_tfidf'] = df['content'].parallel_apply(lambda x: process(str(x)))
      df[["avg_len", "len_text", "len_words", "num_short_w", "per_digit", "per_cap", "f_a", "f_b", "f_c", "f_d", 
          "f_e", "f_f", "f_g", "f_h", "f_i", "f_j", "f_k", "f_l", "f_m", "f_n", "f_o", "f_p", "f_q", "f_r", 
          "f_s", "f_t", "f_u", "f_v", "f_w", "f_x", "f_y", "f_z", "f_0", "f_1", "f_2", "f_3", "f_4", "f_5", "f_6", 
          "f_7", "f_8", "f_9",  "f_e_0", "f_e_1", "f_e_2", "f_e_3", "f_e_4", "f_e_5", "f_e_6", "f_e_7", "f_e_8", 
          "f_e_9", "f_e_10", "f_e_11", "richness"]] = df['content'].parallel_apply(lambda x: extract_style(str(x)))
      df.to_csv("/content/drive/MyDrive/TuringBench_Data/full_AA.csv")  # CHANGE LINK TO WHERE YOU WANT TO SAVE DATA
    else:
      df = pd.read_csv('/content/drive/MyDrive/TuringBench_Data/full_AA.csv')
      df = df.sample(frac=1)

  list_scores = []
  list_f1 = []

  import nltk
  from nltk.util import ngrams
  from collections import Counter
  import heapq

  def return_best_bi_grams(text):
      bigrams = ngrams(text,2)

      data = dict(Counter(bigrams))
      list_ngrams = heapq.nlargest(100, data.keys(), key=lambda k: data[k])
      return list_ngrams

  def return_best_tri_grams(text):
      trigrams = ngrams(text,3)

      data = dict(Counter(trigrams))
      list_ngrams = heapq.nlargest(100, data.keys(), key=lambda k: data[k])
      return list_ngrams

  def find_freq_n_gram_in_txt(text, list_bigram, list_trigram):
      
      to_ret = []
      
      num_bigrams = len(Counter(zip(text,text[1:])))
      num_trigrams = len(Counter(zip(text,text[1:],text[2:])))

      for n_gram in list_bigram: 
          to_ret.append(text.count(''.join(n_gram))/num_bigrams)

      for n_gram in list_trigram: 
          to_ret.append(text.count(''.join(n_gram))/num_trigrams)

      return to_ret

  print('list_senders: ', list_senders)

  for limit in list_senders:

    print("Number of speakers : ", limit)

    # Select top N senders and build Train and Test

    list_spk = list(pd.DataFrame(train['From'].value_counts()[:limit]).reset_index()['index'])
    sub_df = train[train['From'].isin(list_spk)]
    sub_df = sub_df[['From', 'content',  'content_tfidf', "avg_len", "len_text", "len_words", "num_short_w", "per_digit", 
                     "per_cap", "f_a", "f_b", "f_c", "f_d", "f_e", "f_f", "f_g", "f_h", "f_i", "f_j", "f_k", "f_l", "f_m", 
                     "f_n", "f_o", "f_p", "f_q", "f_r", "f_s", "f_t", "f_u", "f_v", "f_w", "f_x", "f_y", "f_z", "f_0", 
                     "f_1", "f_2", "f_3", "f_4", "f_5", "f_6", "f_7", "f_8", "f_9", "f_e_0", "f_e_1", "f_e_2", "f_e_3", 
                     "f_e_4", "f_e_5", "f_e_6", "f_e_7", "f_e_8", "f_e_9", "f_e_10", "f_e_11", "richness"]]
    sub_df = sub_df.dropna()
    
    text = " ".join(sub_df['content'].values)
    list_bigram = return_best_bi_grams(text)
    list_trigram = return_best_tri_grams(text)

    
    sub_df_test = test[test['From'].isin(list_spk)]
    sub_df_test = sub_df_test[['From', 'content',  'content_tfidf', "avg_len", "len_text", "len_words", "num_short_w", "per_digit", 
                                "per_cap", "f_a", "f_b", "f_c", "f_d", "f_e", "f_f", "f_g", "f_h", "f_i", "f_j", "f_k", "f_l", "f_m", 
                                "f_n", "f_o", "f_p", "f_q", "f_r", "f_s", "f_t", "f_u", "f_v", "f_w", "f_x", "f_y", "f_z", "f_0", 
                                "f_1", "f_2", "f_3", "f_4", "f_5", "f_6", "f_7", "f_8", "f_9", "f_e_0", "f_e_1", "f_e_2", "f_e_3", 
                                "f_e_4", "f_e_5", "f_e_6", "f_e_7", "f_e_8", "f_e_9", "f_e_10", "f_e_11", "richness"]]
    sub_df_test = sub_df_test.dropna()
    
    text1 = " ".join(sub_df_test['content'].values)
    list_bigram1 = return_best_bi_grams(text1)
    list_trigram1 = return_best_tri_grams(text1)


    print("Number of texts : ", len(sub_df))

    dict_nlp_enron = {}
    k=0

    for val in np.unique(sub_df.From):
        dict_nlp_enron[val] = k
        k += 1

    sub_df['Target'] = sub_df['From'].apply(lambda x: dict_nlp_enron[x])

    dict_nlp_enron = {}
    k=0
    for val in np.unique(sub_df_test.From):
        dict_nlp_enron[val] = k
        k += 1
    
    sub_df_test['Target'] = sub_df_test['From'].apply(lambda x: dict_nlp_enron[x])


    # ind = train_test_split(sub_df[['content', 'Target']], test_size=0.3, stratify=sub_df['Target'])
    # ind_train = list(ind[0].index)
    # ind_test = list(ind[1].index)

    # nlp_train = sub_df.loc[ind_train]
    # nlp_test = sub_df.loc[ind_test]

    nlp_train = sub_df
    nlp_test = sub_df_test




    # TF-IDF + LR

    print("#####")
    print("Training TF-IDF")

    vectorizer = TfidfVectorizer() #ngram_range=(1,2), max_features=3000
    X_train = vectorizer.fit_transform(nlp_train['content_tfidf'])
    X_test =  vectorizer.transform(nlp_test['content_tfidf'])

    clf = LogisticRegression(random_state=0).fit(X_train, nlp_train['Target'])
    y_pred = clf.predict(X_test)
    score_lr = accuracy_score(nlp_test['Target'], y_pred)
    f1_lr = f1_score(nlp_test['Target'], y_pred, average="macro")
    report = cr(y_true=nlp_test['Target'], y_pred=y_pred, digits=4)
    
    if limit < 11:
      plot_confusion_matrix_from_data(nlp_test['Target'], y_pred)

    print("Training done, accuracy is : ", score_lr)
    print("Training done, f1-score is : ", f1_lr)
    print('classification report: \n', report)

    # Bert + Classification Layer

    print("#####")
    print("Training BERT")

    #if source == "blog":
    #model = ClassificationModel('bert', 'bert-base-cased', limit, config, model_long, tokenizer, args={'reprocess_input_data': True, 'overwrite_output_dir': True}, use_cuda=True) 
    #else:
    model = ClassificationModel('bert', 'bert-base-cased', num_labels=limit, args={'reprocess_input_data': True, 'overwrite_output_dir': True,  'num_train_epochs' : 5}, use_cuda=True) 
    model.train_model(nlp_train[['content', 'Target']])

    predictions, raw_outputs = model.predict(list(nlp_test['content']))
    score_bert = accuracy_score(predictions, nlp_test['Target'])
    f1_bert = f1_score(predictions, nlp_test['Target'], average="macro")
    report = cr(y_true=nlp_test['Target'], y_pred=predictions, digits=4)


    if limit < 11:
      plot_confusion_matrix_from_data(nlp_test['Target'], predictions)

    predictions, raw_out_train = model.predict(list(nlp_train['content']))
    
    print("Training done, accuracy is : ", score_bert)
    print("Training done, f1-score is : ", f1_bert)
    print('classification report: \n', report)

    # Style-based classifier

    print("#####")
    print("Training style classifier")

    X_style_train = nlp_train[["avg_len", "num_short_w", "per_digit", "per_cap", "f_a", "f_b", "f_c", "f_d", "f_e", "f_f", "f_g", "f_h", "f_i", "f_j", 
                               "f_k", "f_l", "f_m", "f_n", "f_o", "f_p", "f_q", "f_r", "f_s", "f_t", "f_u", "f_v", "f_w", "f_x", "f_y", "f_z", "f_0", 
                               "f_1", "f_2", "f_3", "f_4", "f_5", "f_6", "f_7", "f_8", "f_9", "f_e_0", "f_e_1", "f_e_2", "f_e_3", "f_e_4", "f_e_5", 
                               "f_e_6", "f_e_7", "f_e_8", "f_e_9", "f_e_10", "f_e_11", "richness"]]
    X_style_test = nlp_test[["avg_len", "num_short_w", "per_digit", "per_cap", "f_a", "f_b", "f_c", "f_d", "f_e", "f_f", "f_g", "f_h", "f_i", "f_j", 
                             "f_k", "f_l", "f_m", "f_n", "f_o", "f_p", "f_q", "f_r", "f_s", "f_t", "f_u", "f_v", "f_w", "f_x", "f_y", "f_z", "f_0", 
                             "f_1", "f_2", "f_3", "f_4", "f_5", "f_6", "f_7", "f_8", "f_9", "f_e_0", "f_e_1", "f_e_2", "f_e_3", "f_e_4", "f_e_5", 
                             "f_e_6", "f_e_7", "f_e_8", "f_e_9", "f_e_10", "f_e_11", "richness"]]
    
    #clf = xgb.XGBClassifier().fit(X_style_train, nlp_train['Target'])
    clf = LogisticRegression(random_state=0).fit(X_style_train, nlp_train['Target'])
    y_pred = clf.predict(X_style_test)
    y_proba = clf.predict_proba(X_style_test)
    y_proba_train = clf.predict_proba(X_style_train)
    score_style = accuracy_score(nlp_test['Target'], y_pred)
    f1_style = f1_score(nlp_test['Target'], y_pred, average="macro")
    report = cr(y_true=nlp_test['Target'], y_pred=y_pred, digits=4)


    print("Training done, accuracy is : ", score_style)
    print("Training done, f1-score is : ", f1_style)
    print('classification report: \n', report)


    # Model Combination

    print("#####")
    print("Model combination")

    feat_for_BERT_LR_train = np.concatenate([raw_out_train, y_proba_train], axis=1)
    feat_for_BERT_LR_test = np.concatenate([raw_outputs, y_proba], axis=1)

    clf = LogisticRegression(random_state=0).fit(feat_for_BERT_LR_train, nlp_train['Target'])
    #clf = xgb.XGBClassifier().fit(feat_for_BERT_LR_train, nlp_train['Target'])

    y_pred = clf.predict(feat_for_BERT_LR_test)
    score_comb = accuracy_score(nlp_test['Target'], y_pred)
    f1_comb = f1_score(nlp_test['Target'], y_pred, average="macro")
    report = cr(y_true=nlp_test['Target'], y_pred=y_pred, digits=4)


    print("Training done, accuracy is : ", score_comb)
    print("Training done, f1-score is : ", f1_comb)
    print('classification report: \n', report)


    if limit < 11:
      plot_confusion_matrix_from_data(nlp_test['Target'], y_pred) 


    if limit < 11:
      plot_confusion_matrix_from_data(nlp_test['Target'], y_pred)

    # Store scores
    list_scores.append([limit, score_lr, score_bert, score_style, score_comb])
    list_f1.append([limit, f1_lr, f1_bert, f1_style, f1_comb])

  list_scores = np.array(list_scores)

  # Plot the output accuracy
  plt.figure(figsize=(12,8))
  plt.plot(list_scores[:,0], list_scores[:,1], label="TF-IDF + LR")
  plt.plot(list_scores[:,0], list_scores[:,2], label="Bert + Classification layer")
  plt.plot(list_scores[:,0], list_scores[:,3], label="Stylometric")
  plt.plot(list_scores[:,0], list_scores[:,4], label="Bert + Style")
  plt.title("Classification Accuracy depending on the number of speakers")
  plt.xlabel("Number of speakers")
  plt.ylabel("Accuracy")
  plt.legend()
  plt.show()

  return list_scores, list_f1

In [ ]:
list_scores_AA = run_iterations(source="AA", recompute=True)

In [ ]:
# !cp '/content/full_AA.csv' '/content/drive/MyDrive/TuringBench_Data/'